In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score

In [38]:
email_db = pd.read_csv("/content/sample_data/emails.csv", encoding='cp949', sep='\,')
email_db = email_db.drop("Email No.", axis=1) #1열 이메일 인덱스 지움

<ipython-input-38-13512df155a1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  email_db = pd.read_csv("/content/sample_data/emails.csv", encoding='cp949', sep='\,')
Exception ignored in: <function _xla_gc_callback at 0x7f6c2c580040>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [73]:
x = email_db.drop('Prediction', axis=1)
y = email_db['Prediction']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#validation 데이터셋 생성
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

#텐서플로우를 사용하기에, 데이터형 변환을 해줘야 한다.
x_train = tf.convert_to_tensor(x_train, dtype=tf.float64)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float64)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float64)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float64)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float64)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float64)

In [56]:
model = keras.models.Sequential([
    keras.layers.Input(3000), #input layer
    keras.layers.Dense(1800, activation = "relu"),
    keras.layers.Dense(700, activation = "relu"),
    keras.layers.Dense(300, activation = "relu"),
    keras.layers.Dense(100, activation = "relu"),
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(1, activation="sigmoid") #binary classifier이기 때문에 sigmoid로 설정, 다중분류는 softmax
])
model.built

True

In [66]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 1800)              5401800   
                                                                 
 dense_47 (Dense)            (None, 700)               1260700   
                                                                 
 dense_48 (Dense)            (None, 300)               210300    
                                                                 
 dense_49 (Dense)            (None, 100)               30100     
                                                                 
 dense_50 (Dense)            (None, 50)                5050      
                                                                 
 dense_51 (Dense)            (None, 1)                 51        
                                                                 
Total params: 6908001 (26.35 MB)
Trainable params: 690

In [71]:
model.compile(loss = "binary_crossentropy",
              optimizer = "Adam",  #sgd 사용했는데 성능이 별로 안좋아서, adam 사용
              metrics=["accuracy"])

In [72]:
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_val, y_val))

Epoch 1/10
104/104 [==============================] - 3s 8ms/step - loss: 0.4843 - accuracy: 0.9882 - val_loss: 0.1507 - val_accuracy: 0.9771
Epoch 2/10
104/104 [==============================] - 1s 6ms/step - loss: 0.3546 - accuracy: 0.9867 - val_loss: 1.0625 - val_accuracy: 0.9010
Epoch 3/10
104/104 [==============================] - 1s 7ms/step - loss: 0.0275 - accuracy: 0.9949 - val_loss: 0.2159 - val_accuracy: 0.9867
Epoch 4/10
104/104 [==============================] - 1s 8ms/step - loss: 2.7189e-04 - accuracy: 1.0000 - val_loss: 0.6739 - val_accuracy: 0.9879
Epoch 5/10
104/104 [==============================] - 1s 8ms/step - loss: 4.8738e-07 - accuracy: 1.0000 - val_loss: 0.7120 - val_accuracy: 0.9879
Epoch 6/10
104/104 [==============================] - 1s 9ms/step - loss: 1.6586e-08 - accuracy: 1.0000 - val_loss: 0.7418 - val_accuracy: 0.9879
Epoch 7/10
104/104 [==============================] - 1s 8ms/step - loss: 9.9911e-09 - accuracy: 1.0000 - val_loss: 0.7834 - val_accurac

In [59]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

33/33 [==============================] - 0s 3ms/step - loss: 0.4281 - accuracy: 0.9836
Test Loss: 0.42806509137153625
Test Accuracy: 0.9835748672485352


In [60]:
y_pred = model.predict(x_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # 이진 분류의 경우 임계값을 선택하여 이진화
cm = confusion_matrix(y_test, y_pred_binary)

33/33 [==============================] - 0s 2ms/step


In [61]:
cm

array([[729,  10],
       [  7, 289]])

In [65]:
f1 = f1_score(y_test, y_pred_binary)
print(f"F1 Score: {f1}")

F1 Score: 0.9714285714285714
